# Optimize Initial Conditions
## Differential Evolution SciPy

In [1]:
from scipy.optimize import brute
import numpy as np
from datetime import datetime,timedelta
import pandas as pd
from scipy.optimize import differential_evolution


In [2]:
# Load solver
import LearnerICRayNoLoad as L 

In [3]:
def load_confirmed(state, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/confirmados.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[state].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y
    return df2

def load_dead(state, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/mortes.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[state].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y
    return df2

In [4]:
# Initial parameters
dfparam = pd.read_csv("data/param.csv")
dfparam

,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,SP,2020-03-15,200,8000000.0,0,0,800,300,250,100,0.15,0.55,0.01
1,ES,2020-04-01,200,475000.0,0,0,50,250,50,100,0.15,0.65,0.01
2,MG,2020-04-01,200,1100000.0,0,0,200,250,40,100,0.15,0.55,0.01
3,RJ,2020-03-20,200,700000.0,0,0,800,250,50,100,0.15,0.50,0.01
4,CE,2020-03-20,200,800000.0,0,0,800,250,50,100,0.15,0.50,0.01
5,PE,2020-03-20,200,700000.0,0,0,800,250,100,100,0.15,0.50,0.01
6,AM,2020-03-20,200,700000.0,0,0,800,250,100,100,0.15,0.50,0.01


In [5]:
df = pd.read_csv("data/dados_total_estados.csv")
df

,index,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,0,2020-06-06,SE,TOTAL,state,8972,208,74,True,2298696.0,28.0,390.30824,0.0232
1,1,2020-06-05,SE,TOTAL,state,8577,198,73,False,2298696.0,28.0,373.12459,0.0231
2,2,2020-06-04,SE,TOTAL,state,8200,186,72,False,2298696.0,28.0,356.72399,0.0227
3,3,2020-06-03,SE,TOTAL,state,7989,180,71,False,2298696.0,28.0,347.54487,0.0225
4,4,2020-06-02,SE,TOTAL,state,7555,172,70,False,2298696.0,28.0,328.66460,0.0228
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,158210,2020-03-11,BA,TOTAL,state,3,0,5,False,14873064.0,29.0,0.02017,0.0000
2245,158211,2020-03-10,BA,TOTAL,state,2,0,4,False,14873064.0,29.0,0.01345,0.0000
2246,158212,2020-03-09,BA,TOTAL,state,2,0,3,False,14873064.0,29.0,0.01345,0.0000
2247,158213,2020-03-07,BA,TOTAL,state,2,0,2,False,14873064.0,29.0,0.01345,0.0000


In [6]:
from scipy.integrate import odeint
import os
import sys
import io
import ray
import gc
import tensorflow as tf

def create_f(state,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version):
                
    def fobjective(point):
        
        dead=  load_dead(state,date)
        data = load_confirmed(state,date)*(1-ratio)-dead
        
        cleanRecovered=False
        s0, deltaDate, i0, weigthCases, weigthRecov = point
        end_date=datetime.strptime(date, "%Y-%m-%d") + timedelta(days=+int(deltaDate))
                    
        f=L.Learner.remote(state, end_date.strftime("%Y-%m-%d"), predict_range,\
                           s0, e0, a0, i0, r0, d0, startNCases, ratio, weigthCases, weigthRecov, \
                           cleanRecovered, version, data, dead, savedata=True)
        result = f.train.remote() 
        result = ray.get(result) 

        del end_date,cleanRecovered, data, dead         

        gc.collect()

        return result
    return fobjective

In [7]:
@ray.remote(memory=100 * 1024 * 1024)
def opt(state,s0,i0,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version):

    bounds=[(0.8*s0,1.2*s0),(-10,10),(0.8*i0,1.2*i0),\
              (0.3,0.7),(0.001,0.05)]
    f=create_f(state,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version)
    result = differential_evolution(f, bounds)

    return result.x

In [8]:
states=['SP','ES','MG','RJ','PE','CE','AM']
allStates=True
version="1000"
ray.shutdown()
GB=1024*1024*1024
MB=1024*1024
ray.init(object_store_memory=500*MB,memory=2*GB,lru_evict=True,\
         driver_object_store_memory=50*MB,num_cpus=16)
gc.enable()

runID=[]
if allStates:
    for state in states:
    #for districtRegion in DRS:
    #remove previous history file
        strFile='./results/history_'+state+version+'.csv'
        if os.path.isfile(strFile):
            os.remove(strFile)
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
        f_remote=opt.remote(state,s0,i0,e0,a0,r0,d0,startdate,startNCases, \
                                                    ratio, predict_range, version)
        runID.append(f_remote)
else:
    state = "ES" 
    #remove previous history file
    strFile='./results/history_'+state+version+'.csv'
    if os.path.isfile(strFile):
        os.remove(strFile)
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    start_date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    f_remote=opt.remote(state,s0,i0,e0,a0,r0,d0,startdate,startNCases, \
                                                ratio, predict_range, version)
    runID.append(f_remote)



2020-06-12 14:28:25,330	INFO resource_spec.py:212 -- Starting Ray with 1.95 GiB memory available for workers and up to 0.49 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-12 14:28:25,811	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


In [ ]:
optimal=ray.get(runID)

(pid=86733) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=86733)   warnings.warn(warning_msg, ODEintWarning)
(pid=raylet) E0612 14:29:23.021888 86699 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 14: failed to connect to all addresses
(pid=raylet) E0612 14:29:23.022142 86699 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 14: failed to connect to all addresses
(pid=raylet) E0612 14:29:23.022209 86699 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 14: failed to connect to all addresses
(pid=raylet) E0612 14:29:23.022375 86699 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 14: failed to connect to all addresses
(pid=raylet) E0612 14:29:23.022439 86699 node_manager.cc:3537] Failed to send get core worke

(pid=96133) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=96133)   warnings.warn(warning_msg, ODEintWarning)
(pid=101577) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=101577)   warnings.warn(warning_msg, ODEintWarning)
(pid=103262) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=103262)   warnings.warn(warning_msg, ODEintWarning)
(pid=103550) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong D

(pid=117865) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=117865)   warnings.warn(warning_msg, ODEintWarning)
(pid=120839) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=120839)   warnings.warn(warning_msg, ODEintWarning)
(pid=122972) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=122972)   warnings.warn(warning_msg, ODEintWarning)
(pid=raylet) E0612 15:13:28.864188 86699 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 14: failed to connect to all address

(pid=151212) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=151212)   warnings.warn(warning_msg, ODEintWarning)
(pid=770) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=770)   warnings.warn(warning_msg, ODEintWarning)
(pid=307) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=307)   warnings.warn(warning_msg, ODEintWarning)
(pid=1908) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). R

In [ ]:
for i in range(0,len(states)):    

    state=states[i]
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    Date = datetime.strptime(startdate, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(optimal[i][1]))
    dateStr=end_date.strftime("%Y-%m-%d")

    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[i][0]
    dfparam.at[j, "start-date"] = dateStr
    dfparam.at[j, "i0"] = optimal[i][2]
    dfparam.at[j, "WCASES"] = optimal[i][3]
    dfparam.at[j, "WREC"] = optimal[i][4]

    dfparam.to_csv("data/param_optimized_Yabox.csv", sep=",", index=False)
    dfparam
    